In [10]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import os
from os import listdir
from os.path import isfile, join

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [11]:
# parse 3D Intensity measurements
# I - TPX2 spindle intensity

intensity_dir0 = '/Volumes/TOB_WD2/Image_Analysis/PTMs/Analysis/3D_Intensity_measurements/20220804' 
intensity_dir1 = '/Volumes/TOB_WD2/Image_Analysis/PTMs/Analysis/3D_Intensity_measurements/20220921' 
intensity_dir2 = '/Volumes/TOB_WD2/Microscopy/20231025_TPX2/3D_Intensity_TPX2' 

def concat_augmin(folder, staining):
    
    dataframes = []   
    for root, dirs, files in os.walk(folder):
        for file in files:
            if not file.startswith('.'):
                filepath = root + os.sep + file
                if filepath.endswith('Spindle_TPX2.csv'):
                    
                    single_df = pd.read_csv(filepath, encoding = 'latin-1')
                    single_df = single_df[["Average", "Cell_ID", "IntegratedDensity"]]
                    single_df = single_df.rename(columns = 
                                                 {
                                                     "Average": "Average_Spindle_TPX2", 
                                                     "IntegratedDensity": "IntegratedDensity_Spindle_TPX2"
                                                 }
                                                )
                    dataframes.append(single_df) 
                else:
                    pass
            else:
                pass
    df = pd.concat(dataframes)
    return df

augmin_df_0 = concat_augmin(intensity_dir0, 'TPX2')
augmin_df_1 = concat_augmin(intensity_dir1, 'TPX2')
augmin_df_2 = concat_augmin(intensity_dir2, 'TPX2')

augmin_df = pd.concat([augmin_df_0, augmin_df_1, augmin_df_2])

def correct_CellID(x):
    if x.endswith("-out"):
        return x.rsplit("-", 1)[0]
    else:
        return x

augmin_df["Cell_ID"] = augmin_df.Cell_ID.apply(correct_CellID)   
    
print(augmin_df.shape)
augmin_df.head(1)

(332, 3)


Average_Spindle_TPX2                                Cell_ID  \
0              2827.241  20220804_R1E309_PTMs_0X_TPX2_D0_002-1   

   IntegratedDensity_Spindle_TPX2  
0                        44964434

In [12]:
# parse 3D Intensity measurements
# II - Total cell Augmin

Augmin_total_dir0 = '/Volumes/TOB_WD2/Microscopy/20220921_PTMs_MAPs/Total_Cell_TPX2'
Augmin_total_dir1 = '/Volumes/TOB_WD2/Microscopy/20220804_PTMs_MAPs/Total_Cell_TPX2'
Augmin_total_dir2 = '/Volumes/TOB_WD2/Microscopy/20231025_TPX2/Total_Cell_TPX2'

def concat_total(folder, staining):
    average_column = 'Average_Total_' + staining
    minimum_column = 'Minimum_Total_' + staining
    intdent_column = 'IntegratedDensity_Total_' + staining
    
    dataframes = []   
    for root, dirs, files in os.walk(folder):
        for file in files:
            if not file.startswith("."):
                filepath = root + os.sep + file  
                if filepath.endswith('.csv'):
                    single_df = pd.read_csv(filepath, encoding = 'latin-1')
                    single_df = single_df.rename(columns = 
                                                 {
                                                     'Average': average_column, 
                                                     'Minimum': minimum_column,
                                                     'IntegratedDensity': intdent_column
                                                 }
                                                )
                    single_df = single_df[['Cell_ID', average_column, minimum_column, intdent_column]]
                    dataframes.append(single_df)
                else:
                    pass
            else:
                pass
    df = pd.concat(dataframes)
    return df

Augmin_total_0 = concat_total(Augmin_total_dir0, 'TPX2')
Augmin_total_1 = concat_total(Augmin_total_dir1, 'TPX2')
Augmin_total_2 = concat_total(Augmin_total_dir2, 'TPX2')

total_df = pd.concat([Augmin_total_0, Augmin_total_1, Augmin_total_2])

print(total_df.shape)
total_df.head(1)

(347, 4)


Cell_ID  Average_Total_TPX2  \
0  20220921_R1E309_PTMs_0X_TPX2_D0_001-1            1213.762   

   Minimum_Total_TPX2  IntegratedDensity_Total_TPX2  
0                 512                     279431013

In [13]:
# parse Cell Volumes (manual segmentation)

CV_dir_0 = "/Volumes/TOB_WD2/Microscopy/20220804_PTMs_MAPs/Cell_Volumes"
CV_dir_1 = "/Volumes/TOB_WD2/Microscopy/20220921_PTMs_MAPs/Cell_Volumes"
CV_dir_2 = "/Volumes/TOB_WD2/Microscopy/20231025_TPX2/Cell_Volumes"

def concat_CV(folder):
    CV_dataframes = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            if not file.startswith("."):
                filepath = root + os.sep + file  
                if filepath.endswith(".csv"):
                    single_df = pd.read_csv(filepath)
                    single_df = single_df.rename(
                            columns = {
                                "Volume": "Cell_Volume_um3",
                                "SurfaceArea": "Cell_Surface_Area_um2"
                            }
                    )
                    single_df = single_df[["Cell_ID", "Cell_Volume_um3", "Cell_Surface_Area_um2"]]

                    CV_dataframes.append(single_df)
                else:
                    pass
            else:
                pass
    CV_df = pd.concat(CV_dataframes)
    return CV_df 

CV_df_0 = concat_CV(CV_dir_0)
CV_df_1 = concat_CV(CV_dir_1)
CV_df_2 = concat_CV(CV_dir_2)

CV_df = pd.concat([CV_df_0, CV_df_1, CV_df_2])

print(CV_df.shape)
CV_df.head(1)

(348, 3)


Cell_ID  Cell_Volume_um3  \
0  20220804_R1E309_PTMs_0X_TPX2_D0_002-1         2863.157   

   Cell_Surface_Area_um2  
0               1013.712

In [14]:
# parse Spindle3D morphometry
# parse Plugin Measurements

input_dir0 = "/Volumes/TOB_WD2/Microscopy/20220804_PTMs_MAPs/Spindle3D/Correct"
input_dir1 = "/Volumes/TOB_WD2/Microscopy/20220921_PTMs_MAPs/Spindle3D/Correct"
input_dir2 = "/Volumes/TOB_WD2/Microscopy/20231025_TPX2/Spindle3D/Correct"

spindle_input_folders = [
    input_dir0, input_dir1, input_dir2
]

def concat_spindle3d(folderlist):
    dataframes = []
    for folder in folderlist:    
        for root, dirs, files in os.walk(folder):
            for file in files:
                filepath = root + os.sep + file  
                if filepath.endswith(".txt"):
                    single_df = pd.read_csv(filepath, sep = " ", delimiter = "\t", encoding = 'unicode_escape')
                    dataframes.append(single_df)  
                else:
                    pass
    df = pd.concat(dataframes)
    return df

spindle_df = concat_spindle3d(spindle_input_folders)

print("Finished parsing data.")

def get_nakedname(path):
    # parse cell ids from file name in spindle3D table
    base = os.path.basename(str(path))
    naked = os.path.splitext(base)[0]
    return naked

# modify spindle 3d dataframe
spindle_df["Cell_ID"] = spindle_df.Path_InputImage.apply(get_nakedname)

subselection_spindle = [
    "Cell_ID",
    "Spindle_Angle_Degrees",  
    "Spindle_Length_um", 
    "Spindle_Volume_um3", 
    "Spindle_Width_Avg_um", 
    "Tubulin_Spindle_Average_Intensity",
    "Chromatin_Volume_um3",
    "MetaphasePlate_Length_um",
    "MetaphasePlate_Width_um",
    "Version"
]
spindle_df = spindle_df[subselection_spindle]
print(spindle_df.shape)
spindle_df.head(1)

Finished parsing data.
(323, 10)


Cell_ID  Spindle_Angle_Degrees  \
0  20220804_R1E309_PTMs_0X_TPX2_D0_002-1              25.049002   

   Spindle_Length_um  Spindle_Volume_um3  Spindle_Width_Avg_um  \
0          10.574143               248.5              6.921053   

   Tubulin_Spindle_Average_Intensity  Chromatin_Volume_um3  \
0                        8540.720573             394.46875   

   MetaphasePlate_Length_um  MetaphasePlate_Width_um                   Version  
0                      11.5                      3.0  Spindle3D version: 0.8.0

In [16]:
# merge dataframes
df = augmin_df.merge(total_df).merge(CV_df).merge(spindle_df, how = "outer")

print(df.shape)

(324, 17)


In [17]:
# Finalise Dataframe
df["Condition"] = df.Cell_ID.str.split("_").str.get(5)
df["Cells"] = df.Cell_ID.str.split("_").str.get(1)
df["Experiment"] = df.Cell_ID.str.split("_").str.get(0)

# Offset subtraction based on minimum inside the cell binary mask
df["Average_Spindle_TPX2"] = df.Average_Spindle_TPX2 - df.Minimum_Total_TPX2
df["Average_Total_TPX2"] = df.Average_Total_TPX2 - df.Minimum_Total_TPX2

# Fluorescent masses
df["Average_TPX2_Spindle_norm"] = df["Average_Spindle_TPX2"] / df["Average_Total_TPX2"] # TPX2 concentration (average signal) in spindle normalised by total TPX2 concentration (total average signal)
df["TPX2_mass_Spindle_norm"] = df["Average_TPX2_Spindle_norm"] * df["Spindle_Volume_um3"]
df["TPX2_mass_at_Spindle_percent"] = (df["Average_Spindle_TPX2"] * df["Spindle_Volume_um3"]) / (df["Average_Total_TPX2"] * df["Cell_Volume_um3"]) * 100



df["Spindle_Occupancy"] = df.Spindle_Volume_um3 / df.Cell_Volume_um3 * 100

In [18]:
# Export Dataframe

destination = "/Volumes/TOB_WD2/Image_Analysis/TPX2/DataFrames" + "/MasterDataFrame_TPX2.csv"
df.to_csv(destination)
print("Successfully saved dataframe at {}".format(destination))

Successfully saved dataframe at /Volumes/TOB_WD2/Image_Analysis/TPX2/DataFrames/MasterDataFrame_TPX2.csv
